<div align="center">

# YOLOv5 simple object detector

</div>

This Colab notebook demonstrates how to use YOLOv5 for object detection on images and videos with different YOLOv5 variants. YOLOv5 is a state-of-the-art object detection algorithm that provides high accuracy and real-time detection speeds. In this notebook, we'll explore how to use YOLOv5 to detect objects in both static images and videos.


## Importing libraries, module and files

In [ ]:
!git clone https://github.com/mohamedamine99/YOLOv5-object-detection

Cloning into 'YOLOv5-object-detection'...
remote: Enumerating objects: 174, done.
remote: Total 174 (delta 0), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (174/174), 41.77 MiB | 32.55 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [ ]:
import os
import shutil
import time
import random
from collections import namedtuple


import cv2
import matplotlib.pyplot as plt
import numpy as np


# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

In [ ]:
%pip install ultralytics  # install
import ultralytics
ultralytics.checks()  # checks

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.1/107.7 GB disk)


### Setting up paths and working dirs 

In [ ]:
# Set up paths and working directories
coco_names_file = '/content/YOLOv5-object-detection/coco.names'  # Path to file containing COCO object class names

results_path = '/content/results'  # Path to directory where result images will be saved
video_results_path = '/content/video_results'  # Path to directory where result videos will be saved

test_imgs_path = '/content/YOLOv5-object-detection/test imgs'  # Path to directory containing test images
test_vids_path = '/content/YOLOv5-object-detection/test vids'  # Path to directory containing test videos


### loading COCO class names 

In [ ]:
# Reading the COCO dataset 80 class names from the coco names file
labels = []
with open(coco_names_file, 'rt') as coco_file:
    labels = coco_file.read().rstrip('\n').rsplit('\n')
    
print(labels)

# generate random color for each label
colors = []
for _ in labels:
    rand_tuple = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    colors.append(rand_tuple)


['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


### Loading YOLOv5 variants

In [ ]:
# Define a list of YOLOv5 variant model names to be loaded
yolov5_variants_names = ['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l','yolov5x' ]
yolov5_variants_names_6 = ['yolov5n6', 'yolov5s6', 'yolov5m6', 'yolov5l6','yolov5x6' ]

# Create an empty list to hold the loaded models
yolov5_variants = []

# Define a namedtuple to store the PyTorch model and its name
Yolov5_model = namedtuple('Yolov5_model', ['model', 'name'])

# Loop through the list of model names to load each model
for yolo_name in set(yolov5_variants_names + yolov5_variants_names_6):
    # Load the YOLOv5 variant model using torch.hub.load() function from PyTorch
    model = torch.hub.load('ultralytics/yolov5', yolo_name, force_reload=True, verbose = False)
    
    # Create a Yolov5_model object for the loaded model and append it to the list of variants
    y_m = Yolov5_model(model, yolo_name)
    yolov5_variants.append(y_m)

# Print the number of loaded models
print(len(yolov5_variants))


In [ ]:
# creating Directories for Results and Video Results for YOLOv5 Variants
for yolo_name in set(yolov5_variants_names + yolov5_variants_names_6):
    new_dir = os.path.join(results_path, yolo_name)
    os.makedirs(new_dir)

    new_dir = os.path.join(video_results_path, yolo_name)
    os.makedirs(new_dir)

## Object Detection on Images in a Directory Using YOLOv5 Variants




In [ ]:
def run_object_detection_on_images_from_dir(images_path, model_variants,
                                   labels, colors, save_dir,
                                   conf = 0.25,  # NMS confidence threshold
                                   iou = 0.45,  # NMS IoU threshold
                                   classes = None,  # (optional list) filter by class
                                   max_det = 1000 , # maximum number of detections per image
                                   ):
    """
    Run object detection on images from a directory using YOLOv5 models.

    Parameters:
    -----------
    images_path : str
        Path to the directory containing images to be processed.
    model_variants : list
        List of Yolov5_model namedtuples containing YOLOv5 models and their names.
    labels : list
        List of object labels.
    colors : list
        List of colors for each object label.
    save_dir : str
        Path to the directory where the processed images will be saved.
    conf : float, optional
        NMS confidence threshold (default is 0.25).
    iou : float, optional
        NMS IoU threshold (default is 0.45).
    classes : list, optional
        List of classes to filter by (default is None).
    max_det : int, optional
        Maximum number of detections per image (default is 1000).

    Returns:
    --------
    None
    """
    # Loop through all images in the directory
    for img_name in os.listdir(images_path):
        img_path = os.path.join(images_path, img_name)
        img = cv2.imread(img_path)
        print("\n----------------------------")
        print(img_name)
        print(img.shape)

        # Calculate bounding box thickness based on image dimensions
        bbx_thickness = (img.shape[0] + img.shape[1]) // 500

        # Loop through all YOLOv5 models
        for variant in model_variants:
            image = img.copy()
            #print("********************")
            print(variant.name ,end = '  ')

            # Load the YOLOv5 model and set parameters
            model = variant.model
            model.conf = conf
            model.iou = iou
            model.classes = classes
            model.max_det = max_det

            # Make predictions on the image
            results = model(img)
            res = np.array(results.xyxy[0])

            # Loop through all object detection results
            for r in res:

                # Extract object class and confidence score
                score = r[4] * 100
                r = r.astype(int)
                class_id = r[5]

                # Calculate font scale based on object size
                fontScale = (((r[2] - r[0]) / img.shape[0]) + ((r[3] - r[1]) / img.shape[1])) / 2 * 1.5

                # Draw bounding box, a centroid and label on the image
                im = cv2.putText(image,f"{labels[class_id]} {score:,.2f}%" , 
                                (r[0],r[1] - 5), cv2.FONT_HERSHEY_COMPLEX, 
                            fontScale,  colors[class_id], 1, cv2.LINE_AA)
                
                im = cv2.rectangle(im, (r[0],r[1]), (r[2],r[3]), colors[class_id], bbx_thickness)

                center_coordinates = ((r[2] + r[0])//2, (r[3] + r[1]) // 2)
                im =  cv2.circle(im, center_coordinates, 2 , (0,0,255), -1)

            # save resulting images in their corresponding folders
            save_file = os.path.join(save_dir, variant.name, img_name)
            cv2.imwrite(save_file ,im)




In [ ]:
# Object Detection on Images in a Directory Using YOLOv5 Variants

run_object_detection_on_images_from_dir(images_path = test_imgs_path,
                                   model_variants = yolov5_variants,
                                   labels = labels, 
                                   colors = colors, 
                                   save_dir = results_path
                                   )


----------------------------
highway.PNG
(303, 491, 3)
yolov5s6  yolov5x  yolov5n  yolov5l  yolov5m  yolov5s  yolov5x6  yolov5n6  yolov5m6  yolov5l6  
----------------------------
nyc street.PNG
(333, 479, 3)
yolov5s6  yolov5x  yolov5n  yolov5l  yolov5m  yolov5s  yolov5x6  yolov5n6  yolov5m6  yolov5l6  
----------------------------
2 dogs.PNG
(309, 472, 3)
yolov5s6  yolov5x  yolov5n  yolov5l  yolov5m  yolov5s  yolov5x6  yolov5n6  yolov5m6  yolov5l6  
----------------------------
street 2.PNG
(259, 502, 3)
yolov5s6  yolov5x  yolov5n  yolov5l  yolov5m  yolov5s  yolov5x6  yolov5n6  yolov5m6  yolov5l6  
----------------------------
2 cats.PNG
(293, 514, 3)
yolov5s6  yolov5x  yolov5n  yolov5l  yolov5m  yolov5s  yolov5x6  yolov5n6  yolov5m6  yolov5l6  
----------------------------
people crossing the street.jpg
(976, 976, 3)
yolov5s6  yolov5x  yolov5n  yolov5l  yolov5m  yolov5s  yolov5x6  yolov5n6  yolov5m6  yolov5l6  

## Object Detection on Videos in a Directory Using YOLOv5 Variants


In [ ]:
def run_object_detection_on_video(video_path , model, model_name,
                                   labels, colors, save_dir,
                                   conf = 0.25,  # NMS confidence threshold
                                   iou = 0.45,  # NMS IoU threshold
                                   classes = None,  # (optional list) filter by class
                                   max_det = 1000 , # maximum number of detections per image
                                   ):
    
    """
    Runs object detection on a video using a specified model and saves the output video to a specified directory.

    Args:
        video_path (str): path to the input video file
        model (ObjectDetection): object detection model object
        model_name (str): name of the object detection model
        labels (list): list of label names corresponding to the model's classes
        colors (list): list of colors for each class to use for bounding boxes and text labels
        save_dir (str): directory to save the output video file
        conf (float): NMS confidence threshold
        iou (float): NMS IoU threshold
        classes (list): list of classes to filter by (optional)
        max_det (int): maximum number of detections per image (optional)

    Returns:
        None
    """
    
    # Set object detection model properties
    model.conf = conf
    model.iou = iou
    model.classes = classes
    model.max_det = max_det

    # Open input video file
    cap = cv2.VideoCapture(video_path)
    print("\n----------------------------")

    # Get video name 
    vid_name = os.path.basename(video_path)
    print(vid_name, end = ' : ')

    # Get frame dimensions and print information about input video file
    width  = int(cap.get(3) )  # get `width` 
    height = int(cap.get(4) )  # get `height` 
    print((width,height))
    print(video_path)
    print(model_name)

    # Set bounding box thickness based on video dimensions
    bbx_thickness = (height + width) // 500

    # Define output video file
    save_file = os.path.join(save_dir, vid_name[:-4] + ".avi")
    print('saving to :' + save_file)

    # define an output VideoWriter  object
    out = cv2.VideoWriter(save_file,
                        cv2.VideoWriter_fourcc(*"MJPG"),
                        30,(width,height))

    # Check if the video is opened correctly
    if not cap.isOpened():
        print("Error opening video stream or file")

    # Read the video frames
    while cap.isOpened():
        ret, frame = cap.read()

        # If the frame was not read successfully, break the loop
        if not ret:
            print("Error reading frame")
            print((width,height))
            break

        # Run object detection on the frame and calculate FPS
        beg = time.time()
        results = model(frame)
        fps = 1 / (time.time() - beg)

        # Display FPS on frame
        frame = cv2.putText(frame,f"FPS : {fps:,.2f}" , 
                            (5,15), cv2.FONT_HERSHEY_COMPLEX, 
                        0.5,  (0,0,255), 1, cv2.LINE_AA)
        
        # Convert object detection results to numpy array
        results_np = np.array(results.xyxy[0])

        for result in results_np:
            
            score = result[4] * 100
            result = result.astype(int)
            class_id = result[5]

            # Calculate font scale based on object size
            fontScale = (((result[2] - result[0]) / height) + ((result[3] - result[1]) / width)) / 2 * 1.5

            # Draw bounding boxes, text labels, and center points for each detection
            frame = cv2.putText(frame,f"{labels[class_id]} {score:,.2f}%" , 
                            (result[0],result[1] - 5), cv2.FONT_HERSHEY_COMPLEX, 
                        fontScale,  colors[class_id], 1, cv2.LINE_AA)
            
            frame = cv2.rectangle(frame, (result[0],result[1]), (result[2],result[3]), colors[class_id], bbx_thickness)

            center_coordinates = ((result[2] + result[0])//2, (result[3] + result[1]) // 2)

            frame =  cv2.circle(frame, center_coordinates, 2 , (0,0,255), -1)

        # append frame to the video file
        out.write(frame)
        
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # After the loop release the cap 
    cap.release()
    out.release()


In [ ]:
run_object_detection_on_video('/content/YOLOv5-object-detection/test vids/traffic.mp4' ,
                              yolov5_variants[7].model, 
                              yolov5_variants[7].name,
                              labels, colors, results_path,
                              conf = 0.25,  # NMS confidence threshold
                              iou = 0.45,  # NMS IoU threshold
                              classes = None,  # (optional list) filter by class
                              max_det = 1000 , # maximum number of detections per image
                                   )

In [ ]:
# Looping through Test Videos and Running Object Detection on them for Different YOLOv5 Variants.

# Loop through each test video file in the directory
for video_file in os.listdir(test_vids_path):

    # Get the full path of the video file and its name
    video_path = os.path.join(test_vids_path, video_file)
    video_name = os.path.basename(video_path)
    print(video_name)
    print(video_path)

    # Loop through each YOLOv5 variant and run object detection on the video
    for variant in yolov5_variants:
        print(variant.name, end = " - ")
        save_dir = os.path.join(video_results_path, variant.name )

        run_object_detection_on_video(video_path ,
                                    variant.model, 
                                    variant.name,
                                    labels, colors, 
                                    save_dir,
                                    conf = 0.25,  # NMS confidence threshold
                                    iou = 0.45,  # NMS IoU threshold
                                    classes = None,  # (optional list) filter by class
                                    max_det = 1000 , # maximum number of detections per image
                                        )
    
    print("\n-------------")


1935 street scene.mp4
/content/YOLOv5-object-detection/test vids/1935 street scene.mp4
yolov5s6 - 
----------------------------
1935 street scene.mp4 : (1280, 720)
/content/YOLOv5-object-detection/test vids/1935 street scene.mp4
yolov5s6
saving to :/content/video_results/yolov5s6/1935 street scene.avi
Error reading frame
(1280, 720)
yolov5x - 
----------------------------
1935 street scene.mp4 : (1280, 720)
/content/YOLOv5-object-detection/test vids/1935 street scene.mp4
yolov5x
saving to :/content/video_results/yolov5x/1935 street scene.avi
Error reading frame
(1280, 720)
yolov5n - 
----------------------------
1935 street scene.mp4 : (1280, 720)
/content/YOLOv5-object-detection/test vids/1935 street scene.mp4
yolov5n
saving to :/content/video_results/yolov5n/1935 street scene.avi
Error reading frame
(1280, 720)
yolov5l - 
----------------------------
1935 street scene.mp4 : (1280, 720)
/content/YOLOv5-object-detection/test vids/1935 street scene.mp4
yolov5l
saving to :/content/video_

## Saving results

In [ ]:
!zip -r results.zip /content/results

In [ ]:
!zip -r video_results.zip /content/video_results